In [ ]:
# @title
from IPython.display import HTML

HTML("""
<iframe srcdoc='
<div style="
    background: linear-gradient(90deg, #1E3A8A, #06B6D4, #EAB308);
    background-size: 300% 300%;
    text-align: center;
    color: white;
    padding: 35px 40px;
    border-radius: 25px;
    font-family: Poppins, sans-serif;
    font-weight: 800;
    letter-spacing: 3px;
    text-transform: uppercase;
    border: 3px solid #EAB308;
    box-shadow: 0px 0px 30px rgba(234, 179, 8, 0.7);
    margin: 30px 0;
">

<h1 style="font-size: 32px; margin-bottom: 15px;">
📈 AI Powered Stock Price Prediction
</h1>

<p style="font-size: 18px; margin:10px 0;">
✍️ Author: <span style="color:#0B1023; font-weight:800;">Shah Ahmad Noorani</span>
</p>

<div style="margin-top:15px;">

<a href="https://www.linkedin.com/in/shah-ahmad-noorani-5804b0278/" target="_blank">
<img src="https://img.shields.io/badge/LinkedIn-%2300A4CC.svg?style=for-the-badge&logo=LinkedIn&logoColor=white">
</a>

<a href="https://github.com/shahahmad-dev" target="_blank">
<img src="https://img.shields.io/badge/GitHub-%23FF6F61.svg?style=for-the-badge&logo=GitHub&logoColor=white">
</a>

<a href="https://www.kaggle.com/shahahmadnoorani" target="_blank">
<img src="https://img.shields.io/badge/Kaggle-%238a2be2.svg?style=for-the-badge&logo=Kaggle&logoColor=white">
</a>

</div>
</div>
' width="100%" height="260" style="border:none;"></iframe>
""")